In [ ]:
import os
import numpy as np
import arcpy


sde = r'C:\MDGISS\devs\smy2kkl\kkl-db1_giscentral_kkl1.sde'
sdeSamples = 'SDE.SMY_Samples'
arr = arcpy.da.TableToNumPyArray(sdeSamples, ['FOR_NO'])
print(np.unique(arr))


In [ ]:
import os
import arcpy
from datetime import datetime

for_no = 1250

sde = r'C:\MDGISS\devs\smy2kkl\kkl-db1_giscentral_kkl1.sde'
sdeSamples = 'SDE.SMY_Samples'
sdeStands = 'SDE.SMY_Stands'

arcpy.env.workspace = sde

appenddate = datetime.today().date().strftime('%d/%m/%Y')

# 1. UpdateCursor

with arcpy.da.Editor(sde) as edit:
    for sde_layer in [sdeStands, sdeSamples ]:
        try:
            print(sde_layer)
            fields = ['FOR_NO', 'AppendDate']
            expr = f'FOR_NO = {for_no}'
            with arcpy.da.UpdateCursor(sde_layer, fields, expr) as uCur:
                for row in uCur:
                    print(row[0]) # = appenddate
                    row[1] = datetime.today().date()
                    uCur.updateRow(row)
            del uCur
        except TypeError as e:
            if 'cannot update the table' in str(e):
                print ('Error cannot update the table. Tring "CalculateField"')
                # 2. CalculateField
                lyr = 'xxlyr'                   
                arcpy.management.MakeFeatureLayer(sde_layer, lyr,  where_clause=f'FOR_NO = {for_no}')
                arcpy.management.CalculateField(lyr, 'AppendDate', f'"{appenddate}"', "PYTHON3")
                del lyr


In [ ]:
######################################################
#   Calculate AppendDate values in SDE layer
######################################################

import os
import arcpy
from datetime import datetime

for_no = 4298
# with arcpy.da.UpdateCursor(sdeObject, fields, expr) as uCur:
#     for row in uCur:
#         row[1] = today
#         uCur.updateRow(row)
# print('Finished')

sde = r'C:\MDGISS\devs\smy2kkl\kkl-db1_giscentral_kkl1.sde'
sdeSamples = 'SDE.SMY_Samples'
sdeStands = 'SDE.SMY_Stands'

arcpy.env.workspace = sde
appenddate = datetime.today().date()

# arcpy.management.CalculateField(
#     in_table="SMY_Stands",
#     field="AppendDate",
#     expression="'28/08/2024'",
#     expression_type="PYTHON3",
#     code_block="",
#     field_type="TEXT",
#     enforce_domains="NO_ENFORCE_DOMAINS"
# )
with arcpy.da.Editor(sde) as edit:
    for sde_layer in [sdeSamples, sdeStands]:
        print(sde_layer)
        fields = ['FOR_NO', 'AppendDate']
        expr = f'FOR_NO = {for_no}'
        with arcpy.da.UpdateCursor(sde_layer, fields, expr) as uCur:
            for row in uCur:
                row[1] = appenddate
                uCur.updateRow(row)
        del uCur
        #arcpy.AddMessage (f'The date {appenddate} added AppendDate')
        print(f'The date {appenddate} added to AppendDate field in {sde_layer}')

In [ ]:
###################################################
#  Delete all data of forest using cursor from SDE
###################################################
import os
import arcpy

for_no = 1250

ds_dict = {'SDE.SMY_Samples':['SDE.MY_Samples__ATTACH', 'SDE.SMY_SamplesInvasiveSpecies', 'SDE.SMY_SamplesSpecies',
        'SDE.SMY_SamplesUnderstory', 'SDE.SMY_SamplesVitalForest'],
        'SDE.SMY_Stands':['SDE.SMY_StandsInvasiveSpecies', 'SDE.SMY_StandsSpecies',
        'SDE.SMY_StandsUnderstory', 'SDE.SMY_StandsVitalForest']}

layer_flds = ['FOR_NO', 'GlobalID']

sde = r'C:\MDGISS\devs\smy2kkl\kkl-db1_giscentral_kkl1.sde'
arcpy.env.workspace = sde
edit = arcpy.da.Editor(sde)
edit.startEditing()
edit.startOperation()

for layer in ds_dict:
    exp = f'FOR_NO = {for_no}'
    with arcpy.da.UpdateCursor(layer, layer_flds, exp) as upd_cursor:
        for row in upd_cursor:
            gid = row[1]
            print(row[1])
            upd_cursor.deleteRow()
            
            for tab in ds_dict[layer]:
                if layer == 'SDE.SMY_Samples':
                    if tab == 'SDE.SMY_Samples__ATTACH': 
                        rfld = 'REL_GLOBALID'
                    else:
                        rfld = 'parentglobalid'                   
                elif layer == 'SDE.SMY_Stands':   
                    rfld = 'stand_ID'
                tab_exp = f"{rfld} = '{gid}'"
                with arcpy.da.UpdateCursor(tab, [rfld], tab_exp) as del_cursor:
                    for row in del_cursor:
                        del_cursor.deleteRow()
                
            
edit.stopOperation()
edit.stopEditing(True)


In [ ]:
#################################################
#  Delete all data of forest using cursor from fgdb
#################################################
import os
import arcpy

for_no = 1311

ds_dict = {'SMY_Samples':['SMY_Samples__ATTACH', 'SMY_SamplesInvasiveSpecies',
            'SMY_SamplesSpecies','SMY_SamplesUnderstory',
            'SMY_SamplesVitalForest'],
            'SMY_Stands':['SMY_StandsInvasiveSpecies', 'SMY_StandsSpecies',
            'SMY_StandsUnderstory', 'SMY_StandsVitalForest']}

layer_flds = ['FOR_NO', 'GlobalID']

ws = r'C:\MDGISS\devs\smy2kkl\smy2kkl.gdb'
arcpy.env.workspace = ws
edit = arcpy.da.Editor(ws)
edit.startEditing()
edit.startOperation()

for layer in ds_dict:
    exp = f'FOR_NO = {for_no}'
    with arcpy.da.UpdateCursor(layer, layer_flds, exp) as upd_cursor:
        for row in upd_cursor:
            gid = row[1]
            print(row[1])
            upd_cursor.deleteRow()
            
            for tab in ds_dict[layer]:
                if layer == 'SMY_Samples':
                    if tab == 'SMY_Samples__ATTACH': 
                        rfld = 'REL_GLOBALID'
                    else:
                        rfld = 'parentglobalid'                   
                elif layer == 'SMY_Stands':    
                    rfld = 'stand_ID'
                tab_exp = f"{rfld} = '{gid}'"
                with arcpy.da.UpdateCursor(tab, [rfld], tab_exp) as del_cursor:
                    for row in del_cursor:
                        del_cursor.deleteRow()
                
            
edit.stopOperation()
edit.stopEditing(True)


In [ ]:
#############################
#  Update domain
#############################
import os
import arcpy
import pandas as pd

def fromExcel(xlsfile, sheet_name):
    df = pd.read_excel(xlsfile, sheet_name)
    return df

def create_domain(df,domName, ftype, gdb):
    try:
        # Get domain data
        codeField, descField = ['Code', 'Description']
        ddf = df[df['Name']== domName][[codeField, descField]]
        arcpy.management.CreateDomain(gdb, domName, domName, ftype, "CODED")

        for row in ddf.itertuples(False):
            print(row[0], row[1])
            code = row[0]
            val = row[1]
            arcpy.management.AddCodedValueToDomain(gdb, domName, code, val)

    except Exception as e:
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print(f'{exc_type} {fname} {exc_tb.tb_lineno}')
        print (e)

    
sde = r'C:\MDGISS\devs\smy2kkl\kkl-db1_giscentral_kkl1.sde'
xlsfile = r'C:\MDGISS\devs\smy2kkl\AppData\smy_data_design_template.xlsx'
sht = 'SDE Domains'


domName = 'smySpeciesList'
ftype = 'SHORT'
names = [
        'SDE.SMY_Samples', 'SDE.SMY_Stands',
        'SDE.SMY_SamplesInvasiveSpecies', 'SDE.SMY_SamplesSpecies',
        'SDE.SMY_SamplesUnderstory', 'SDE.SMY_SamplesVitalForest',
        'SDE.SMY_StandsInvasiveSpecies', 'SDE.SMY_StandsSpecies',
        'SDE.SMY_StandsUnderstory', 'SDE.SMY_StandsVitalForest'
        ]

tab_fields = []
arcpy.env.workspace = sde

# Remove domain from field(s) in table
for name in names:
    for f in  arcpy.ListFields(name):
        if f.domain == domName:
            print(name, f.name)
            tab_fields.append((name, f.name))
            arcpy.management.RemoveDomainFromField(name, domName)

# Delete domain
arcpy.management.DeleteDomain(sde, domName)

# Create a new domain
df = fromExcel(xlsfile, sht)
create_domain(df, domName, ftype, sde)

# Assign domain to field
for t, f in tab_fields:
    arcpy.management.AssignDomainToField(t, f, domName)
    print(t,f)
    
print('Finished')

In [ ]:
###########################################################
#  Find code value in fields with given domain
###########################################################
import os
import arcpy

layer = r'C:\MDGISS\devs\smy2agol\samples2agol.gdb\SMY_SamplesInvasiveSpecies'
dom = 'smySpeciesList'
code = 2394
flds = ['OID@']
oids = []
for f in  arcpy.ListFields(layer):
    if f.domain == dom:
        flds.append(f.name)
print(flds)
with arcpy.da.SearchCursor(layer, flds) as sCur:
    for row in sCur:
        for i, v in enumerate(row[1:]):
            if v == code:
                oids.append(row[0])  
print(oids)


In [ ]:
###########################################################
#       Check domains associated with SMY data
###########################################################
import os
import arcpy
sde = r'C:\MDGISS\devs\smy2kkl\kkl-db1_giscentral_kkl1.sde'
arcpy.env.workspace = sde
names = ['SDE.SMY_Samples', 'SDE.SMY_Stands']

for name in names:
    print (name)
    sde_layer = os.path.join(sde, name)
    for f in  arcpy.ListFields(sde_layer):
        if f.domain != '' and pattern in f.domain:
            print('\t', f.name, f.domain)
        
    desc= arcpy.da.Describe(sde_layer)
    for rc in desc['relationshipClassNames']:
        rc_desc = arcpy.da.Describe(rc)
        t = rc_desc['destinationClassNames'][0]
        print(t)
        for f in  arcpy.ListFields(t):
            if f.domain != '' and pattern in f.domain:
                print('\t', f.name, f.domain)

print('Finished')
            
    
    

In [ ]:
########################################
#     Update value of code species
########################################
import os
import pandas as pd
import arcpy

sde = r'C:\MDGISS\devs\smy2kkl\kkl-db1_giscentral_kkl1.sde'
gdb = r'C:\MDGISS\devs\smy2kkl\smy2kkl.gdb'
xlsfile = r'C:\MDGISS\devs\smy2kkl\AppData\smy_data_design_template.xlsx'
sht = 'newcodes'
dom = 'smyCoverType'
layername = 'SDE.SMY_Stands'

df = pd.read_excel(xlsfile, sht)
code_dic = {}
for old, new in df[df['SDE_DOMAIN']== dom][['OLDCODE', 'CODE']].itertuples(False):
    code_dic[old] = new
    
oldcode_list = sorted(list(code_dic.keys()))

layer = os.path.join(sde, layername)
print(layer)
#layer = os.path.join(gdb, layername)
fields = arcpy.ListFields(layer)
dom_flds = []
for f in  fields:
    if f.domain == dom:
        dom_flds.append(f.name)
print(dom_flds)        

edit = arcpy.da.Editor(sde)
edit.startEditing()
edit.startOperation()
arcpy.env.workspace = sde
with arcpy.da.UpdateCursor(layername, dom_flds) as uCur:
    for row in uCur: 
        if row[1] != None:
            if row[1] in oldcode_list:
                row[1] = code_dic[row[1]]
                uCur.updateRow(row)
edit.stopOperation()
edit.stopEditing(True)
                
print('Finished')

In [ ]:
#########################################
#    Add or overwrite domain in SDE
#########################################
import os
import pandas as pd
import arcpy
    
def create_domain(df,domName, ftype, gdb):
    try:
        # Get domain data
        codeField, descField = ['Code', 'Description']
        ddf = df[df['Name']== domName][[codeField, descField]]
        arcpy.management.CreateDomain(gdb, domName, domName, ftype, "CODED")

        for row in ddf.itertuples(False):
            print(row[0], row[1])
            code = row[0]
            val = row[1]
            arcpy.management.AddCodedValueToDomain(gdb, domName, code, val)

    except Exception as e:
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print(f'{exc_type} {fname} {exc_tb.tb_lineno}')
        print (e)
    
def create_domains(df, gdb):
    try:
        # Get list of domains
        domains = df[['Domain_Name', 'DataType']].drop_duplicates()
        codeField, descField = ['Code', 'Description']
        for row in domains.itertuples(False):
            domName = row[0]
            if domName != 'CovtypeSpeciesProportion':
                print(domName)            
                ftype = row[1]
                arcpy.management.CreateDomain(gdb, domName, domName, ftype, "CODED")
                ddf = df[df['Domain_Name']== domName][[codeField, descField]]
                for row in ddf.itertuples(False):
                    print(row[0], row[1])
                    code = row[0]
                    val = row[1]
                    arcpy.management.AddCodedValueToDomain(gdb, domName, code, val)

    except Exception as e:
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print(f'{exc_type} {fname} {exc_tb.tb_lineno}')
        print (e)

def fromExcel(xlsfile, sheet_name):
    df = pd.read_excel(xlsfile, sheet_name)
    return df



sde = r'C:\MDGISS\devs\smy2kkl\kkl-db1_giscentral_kkl1.sde'
xlsfile = r'C:\MDGISS\devs\smy2kkl\AppData\smy_data_design_template.xlsx'
sht = 'SDE Domains'
domName = 'smyVegformLayer'
ftype = 'SHORT'
df = fromExcel(xlsfile, sht)
print(df.columns)
                                              
domains = arcpy.da.ListDomains(sde)

if not domName in [d.name for d in domains]:
    create_domain(df, domName, ftype, sde)
print('Finished ')

In [ ]:
# Get fields by domain

import os
import arcpy

ws = r'C:\MDGISS\devs\smy2kkl\kkl-db1_giscentral_kkl1.sde'
fc = 'SDE.SMY_Stands'
dom = 'smy'


In [ ]:
######################################################
#  Checking values of fields with associated domains
######################################################
import os
import pandas as pd
import arcpy

def value_list(layer, field):
    arr = arcpy.da.TableToNumPyArray(layer, [field], null_value=-9999)
    df = pd.DataFrame(data=arr).reset_index()
    return sorted(list(df[df[field] != -9999][field].drop_duplicates().values))

def alldomains2dic(ws):
    dom_dic = {}
    all_domains = arcpy.da.ListDomains(ws)
    for d in all_domains:
        if d.name[0:3] == 'smy':
            klist = []
            for k,v in d.codedValues.items():
                klist.append(k)
            dom_dic[d.name] =  klist
    return dom_dic

#############################################################
#############################################################

ws = r'C:\MDGISS\devs\smy2kkl\kkl-db1_giscentral_kkl1.sde'
ws = r'C:\MDGISS\devs\smy2agol\smy2agol.gdb'
layer = 'SMY_Stands'

dom_dic = alldomains2dic(ws)

dom_flds = [(f.name , f.domain) for f in arcpy.ListFields(layer) if f.domain ]
for fname, fdom in dom_flds:
    fld_values = value_list(layer, fname)
    dom_codes = dom_dic[fdom]
    diff = set(fld_values) - set(dom_codes)
    if len(diff)> 0:
        print(fname, fdom, diff )

print('Finished')
#

In [ ]:
##########################################
# Add current date to AppendDate field of
# SDE smy layer
##########################################

from datetime import datetime
import arcpy

sde_smy_layer = 'SMY_Stands'
date_fld = 'AppendDate'
sde = r'C:\MDGISS\devs\smy2kkl\kkl-db1_giscentral_kkl1.sde'

with arcpy.da.Editor(sde) as edit:
    with arcpy.da.UpdateCursor(sde_smy_layer, [date_fld]) as uCur:
        for row in uCur:
            uCur.updateRow([datetime(2024, 4,1)])
print('Finished')

In [ ]:
import arcpy

# Use a connection file to create the connection
egdb =r'C:\MDGISS\devs\smy2kkl\kkl-db1_giscentral_kkl1.sde'
egdb_conn = arcpy.ArcSDESQLExecute(egdb)

table_name = 'SDE.SMY_TEST'
field_name = 'PercentByTen'

sql = f'DELETE FROM {table_name} WHERE {field_name}=10;'


egdb_return = egdb_conn.execute(sql)
# for i in egdb_return:
#     print('{}: {}'.format(*i))

In [ ]:
# 
import os
import pandas as pd
import arcpy

def forest_list(layer):
    fields = [ 'FOR_NO'] #, 'HELKA', 'STAND_NO', 'GlobalID']
    arr = arcpy.da.TableToNumPyArray(layer, fields, null_value=-1)
    df = pd.DataFrame(data=arr).reset_index()
    return sorted(list(df['FOR_NO'].drop_duplicates().values))

sde = r'C:\MDGISS\devs\smy2kkl\kkl-db1_giscentral_kkl1.sde'
stands_layer_name = 'SDE.SMY_Stands'
samples_layer_name = 'SDE.SMY_Samples'
stands_layer = os.path.join(sde, stands_layer_name)
samples_layer = os.path.join(sde, samples_layer_name)

forno_stands = forest_list(stands_layer)
print(len(forno_stands), forno_stands)
# forno_samples =  forest_list(samples_layer)
# print(set(forno_samples) - set(forno_stands))

#


In [ ]:
############################################
# Delete all data from SMY layers and tables
############################################

import os
import pandas as pd
import arcpy

sde_file = 'kkl-db1_giscentral_kkl1.sde'

aprx = 'CURRENT'
p = arcpy.mp.ArcGISProject(aprx)
proj_dir = p.homeFolder

sde = os.path.join(proj_dir, sde_file)

arcpy.env.workspace = sde

for l in arcpy.ListFeatureClasses('*SMY*'):
    cnt = int(arcpy.management.GetCount(l)[0])
    if cnt > 0:
        arcpy.management.DeleteFeatures(l)
        print(f'{cnt} features deleted from {l} feature class')
    else:
        print (f'There is no data in {l} layer')
        
for t in arcpy.ListTables('*SMY*'):
    cnt = int(arcpy.management.GetCount(t)[0])
    if cnt > 0:
        arcpy.management.DeleteRows(t)
        print(f'{cnt} rows deleted from {t} table')
    else:
        print (f'There is no data in {t} table')


In [ ]:
########################################################
# Delete existing ralationship classes for SMY data in SDE
########################################################
import os
import pandas as pd
import arcpy

sde_file = 'kkl-db1_giscentral_kkl1.sde'

aprx = 'CURRENT'
p = arcpy.mp.ArcGISProject(aprx)
proj_dir = p.homeFolder

sde = os.path.join(proj_dir, sde_file)
appdata_dir = os.path.join(proj_dir,'AppData')

arcpy.env.workspace = sde

# Get relationship classes data
rc_df = pd.read_excel(ddfile, 'SDE RC')

for root, dirs, rcs in arcpy.da.Walk(datatype='RelationshipClass'):
    for rc in rcs:
        if 'RC' in rc:
            arcpy.management.Delete(rc)
            print(rc)


In [ ]:
########################################################
# Create a new ralationship classes for SMY data in SDE
########################################################

import os
import pandas as pd
import arcpy

sde_file = 'kkl-db1_giscentral_kkl1.sde'

aprx = 'CURRENT'
p = arcpy.mp.ArcGISProject(aprx)
proj_dir = p.homeFolder

sde = os.path.join(proj_dir, sde_file)
appdata_dir = os.path.join(proj_dir,'AppData')

# Get design data 
ddfile= os.path.join(appdata_dir, 'smy_data_design_template.xlsx')

#gdb = os.path.join(data_dir, f'smy{for_no}.gdb')
arcpy.env.workspace = sde

# Get relationship classes data
rc_df = pd.read_excel(ddfile, 'SDE RC')

for root, dirs, rcs in arcpy.da.Walk(datatype='RelationshipClass'):
    for rc in rcs:
        if 'RC' in rc:
            arcpy.management.Delete(rc)
            print(f' The relationship class {rc} deleted.')

for row in rc_df.itertuples(False):
    print(row.name)
    if arcpy.Exists(row.destinationClassNames):        
        arcpy.management.CreateRelationshipClass(row.originClassNames,
                                                 row.destinationClassNames,
                                                 row.name,
                                                 row.rcType,
                                                 row.forwardPathLabel,
                                                 row.backwardPathLabel,
                                                 row.notification,
                                                 row.cardinality,
                                                 'NONE',
                                                 row.originPrimaryKey,
                                                 row.originForeignKey)

        # Set split policy
        if row.originClassNames == 'stands' and row.destinationClassNames != 'samples':
            arcpy.management.SetRelationshipClassSplitPolicy(row.name, 
                                            "DUPLICATE_RELATED_OBJECTS")
    else:
        print(f'The {row.destinationClassNames} does not exist.')


In [ ]:
# Check relationship in SDE
import os
import pandas as pd
import arcpy

rc_sde_dic ={'SDE.SMY_Samples':['SDE.SMY_SamplesInvasiveSpecies', 'SDE.SMY_SamplesUnderstory',                            
              'SDE.SMY_SamplesSpecies', 'SDE.SMY_SamplesVitalForest'],
             'SMY_Stands':['SMY_StandsInvasiveSpecies','SMY_StandsUnderstory',
             'SMY_StandsSpecies','SMY_StandsVitalForest']}

sde = 'C:\MDGISS\devs\smy2kkl\kkl-db1_giscentral_kkl1.sde'
layer_name = 'SDE.SMY_Samples'
layer = os.path.join(sde, layer_name)

fields = [ 'FOR_NO', 'HELKA', 'STAND_NO', 'GlobalID']
arr = arcpy.da.TableToNumPyArray(layer, fields, null_value=-1)
samples_df = pd.DataFrame(data=arr)

# Origin key field in smy samples
primary_key = 'GlobalID'
foreign_key = 'ParentGlobalID'

arcpy.env.workspace = sde
frames = []
for tab in rc_sde_dic[layer_name]:
    arr = arcpy.da.TableToNumPyArray(tab, ['OID@', foreign_key]) #, null_value=null_dict)
    tab_df = pd.DataFrame(data=arr )
    merge_df = pd.merge(samples_df, tab_df, how='outer', left_on = primary_key, right_on=foreign_key)
    #grp = merge_df['FOR_NO'].groupby(['FOR_NO']).count()
    grp = merge_df[['FOR_NO','GlobalID']].groupby(['FOR_NO']).count()
    print(grp)
    grp['Table'] = tab
    frames.append(grp.reset_index())
all_df = pd.concat(frames)

out_csv = r'C:\MDGISS\devs\smy2kkl\Output\sde_check_rc.csv'
# out_df = samples_df.merge(all_df, left_on=primary_key, right_on=foreign_key, how='left')
# out_df['UID'] = out_df['GlobalID'] + ':' + out_df['TID']
all_df.to_csv(out_csv, index=False)